In [ ]:
import sqlite3, json
import pandas as pd

# 1. Connect to the database
conn = sqlite3.connect('valorant_esports.db')

teams_q =  "SELECT * FROM teams"
teams_table = pd.read_sql_query(teams_q, conn)

series_q =  "SELECT series_id, tournament_name, format_name, start_date FROM series"
series_table = pd.read_sql_query(series_q, conn)

conn.close()

In [14]:
teams_table.info()

<class 'pandas.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   index                44 non-null     int64
 1   series_id            44 non-null     int64
 2   format               44 non-null     str  
 3   team_id              44 non-null     str  
 4   name                 44 non-null     str  
 5   won                  44 non-null     int64
 6   kills                44 non-null     int64
 7   score                44 non-null     int64
 8   killAssistsReceived  44 non-null     int64
 9   killAssistsGiven     44 non-null     int64
 10  selfkills            44 non-null     int64
 11  firstKill            44 non-null     int64
 12  weaponKills          44 non-null     str  
 13  teamkills            44 non-null     int64
 14  deaths               44 non-null     int64
 15  player_ids           44 non-null     str  
 16  player1              44 non-null     st

In [ ]:
def get_series_details(series_id):
    conn = sqlite3.connect('valorant_esports.db')
    series_q = f"SELECT series_id, tournament_name, format_name, start_date FROM series WHERE series_id = '{series_id}'"
    series_table = pd.read_sql_query(series_q, conn)
    conn.close()
    
    if series_table.empty:
        return {}
    
    series_details = series_table.iloc[0].to_dict()
    return series_details

def get_team_ids():
    conn = sqlite3.connect('valorant_esports.db')
    teams_q =  "SELECT * FROM teams"
    teams_table = pd.read_sql_query(teams_q, conn)
    conn.close()
    return teams_table[['team_id', 'name']].to_dict('records')

def get_team_series_ids(team_id):
    conn = sqlite3.connect('valorant_esports.db')
    teams_q =  "SELECT * FROM teams"
    teams_table = pd.read_sql_query(teams_q, conn)
    conn.close()
    teams_df = teams_table[teams_table['team_id'].astype(str) == str(team_id)]
    series_ids = teams_df['series_id'].tolist()
    series_details = []
    for series_id in series_ids:
        series_detail = get_series_details(series_id)
        series_details.append({
            'series_id': series_detail['series_id'],
            'tournament_name': series_detail['tournament_name'],
            "start_date" : series_detail['start_date'],
            "format_name" :series_detail['format_name'],
        })
    return series_details

def get_player_ids_of_team_series(series_id, team_id):
    conn = sqlite3.connect('valorant_esports.db')
    teams_q =  "SELECT * FROM 'all-players'"
    players_table = pd.read_sql_query(teams_q, conn)
    conn.close()
    players_df = players_table[(players_table['series_id'].astype(str) == str(series_id)) & (players_table['team_id'].astype(str) == str(team_id))]
    return players_df[['player_id', 'player_name']].to_dict('records')


# get_team_ids()
# get_team_series_ids(team_id= 79)
get_player_ids_of_team_series(team_id= 79, series_id=2819694)
# get_team_series_ids(teams_table=teams_table, team_id= 79, series_id=2819694)

[{'player_id': '10612', 'player_name': 'v1c'},
 {'player_id': '10636', 'player_name': 'OXY'},
 {'player_id': '1195', 'player_name': 'mitch'},
 {'player_id': '1193', 'player_name': 'Xeppaa'},
 {'player_id': '2348', 'player_name': 'neT'},
 {'player_id': '10612', 'player_name': 'v1c'},
 {'player_id': '10636', 'player_name': 'OXY'},
 {'player_id': '1195', 'player_name': 'mitch'},
 {'player_id': '1193', 'player_name': 'Xeppaa'},
 {'player_id': '2348', 'player_name': 'neT'}]

In [21]:
import pandas as pd
import sqlite3
import json

def get_series_details(team_id):
    conn = sqlite3.connect('valorant_esports.db')
    series_q = f"""
        SELECT s.series_id, s.tournament_name, s.format_name, s.start_date, ts.won
        FROM series s
        JOIN 'all-teams' ts ON s.series_id = ts.series_id
        WHERE ts.team_id = '{team_id}'
    """
    series_table = pd.read_sql_query(series_q, conn)
    conn.close()
    
    if series_table.empty:
        return {}
    
    series_details = series_table.to_dict(orient='records')
    return series_details

# Example usage:
team_id = '81'
s = get_series_details(team_id)
s

[{'series_id': '2629392',
  'tournament_name': 'VCT Americas - Kickoff 2024 (Groups: Group B)',
  'format_name': 'best-of-3',
  'start_date': '2024-02-17',
  'won': 0},
 {'series_id': '2637962',
  'tournament_name': 'VCT Americas - Kickoff 2024 (Play-Ins: Play In Groups)',
  'format_name': 'best-of-3',
  'start_date': '2024-02-26',
  'won': 0},
 {'series_id': '2648624',
  'tournament_name': 'VCT Masters - Masters Madrid (Swiss: Swiss)',
  'format_name': 'best-of-3',
  'start_date': '2024-03-14',
  'won': 0},
 {'series_id': '2648633',
  'tournament_name': 'VCT Masters - Masters Madrid (Swiss: Swiss)',
  'format_name': 'best-of-3',
  'start_date': '2024-03-18',
  'won': 1},
 {'series_id': '2648633',
  'tournament_name': 'VCT Masters - Masters Madrid (Swiss: Swiss)',
  'format_name': 'best-of-3',
  'start_date': '2024-03-18',
  'won': 1},
 {'series_id': '2648639',
  'tournament_name': 'VALORANT Masters - Masters Madrid (Playoffs: Playoffs)',
  'format_name': 'best-of-5',
  'start_date': '

In [14]:
def get_team_players(team_id):
    conn = sqlite3.connect('valorant_esports.db')
    players_q = f"""
        SELECT DISTINCT p.player_id, p.player_name
        FROM 'all-players' p
        WHERE p.team_id = '{team_id}'
    """
    players_table = pd.read_sql_query(players_q, conn)
    conn.close()
    
    if players_table.empty:
        return {}
    
    players_details = players_table.to_dict(orient='records')
    return players_details

team_id = '81'
s = get_team_players(team_id)
len(s), s

(5,
 [{'player_id': '14', 'player_name': 'aspas'},
  {'player_id': '10630', 'player_name': 'Verno'},
  {'player_id': '2837', 'player_name': 'artzin'},
  {'player_id': '3320', 'player_name': 'cortezia'},
  {'player_id': '629', 'player_name': 'xenom'}])

In [ ]:
def get_team_players(team_id):
    conn = sqlite3.connect('valorant_esports.db')
    players_q = f"""
        SELECT DISTINCT p.player_id, p.player_name
        FROM 'all-players' p
        WHERE p.team_id = '{team_id}'
    """
    players_table = pd.read_sql_query(players_q, conn)
    conn.close()
    
    if players_table.empty:
        return {}
    
    players_details = players_table.to_dict(orient='records')
    return players_details

team_id = '81'
s = get_team_players(team_id)
len(s), s